In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# pd.set_option('display.max_rows', 5000)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

## Reading the data

In [3]:
data = pd.read_csv('data/pollution_us_2000_2016.csv')

## Some elementary analysis

In [4]:
data.columns

Index(['Unnamed: 0', 'State Code', 'County Code', 'Site Num', 'Address',
       'State', 'County', 'City', 'Date Local', 'NO2 Units', 'NO2 Mean',
       'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', 'O3 Units',
       'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Units',
       'SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI',
       'CO Units', 'CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI'],
      dtype='object')

In [5]:
data.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [6]:
data.nunique()

Unnamed: 0           134576
State Code               47
County Code              73
Site Num                110
Address                 204
State                    47
County                  133
City                    144
Date Local             5996
NO2 Units                 1
NO2 Mean              31859
NO2 1st Max Value       990
NO2 1st Max Hour         24
NO2 AQI                 129
O3 Units                  1
O3 Mean                8196
O3 1st Max Value        134
O3 1st Max Hour          24
O3 AQI                  125
SO2 Units                 1
SO2 Mean              12736
SO2 1st Max Value       921
SO2 1st Max Hour         24
SO2 AQI                 140
CO Units                  1
CO Mean               34123
CO 1st Max Value       2698
CO 1st Max Hour          24
CO AQI                  107
dtype: int64

#### Observing the stats about data up till here, I found the follwoing:
    - I should drop Unnamed: 0. This is just an index column.
    
    - The NO2 units, SO2 units, O3 units and CO units have no significance.
        - They only have a single unique value which is "Parts per billion"
        - I will drop these too as they would require a lot of memory.
    
    - NO2 and SO2 have Parts per Billion.
    - O3 and CO have Parts per Million.

In [7]:
data = data.drop(['Unnamed: 0','NO2 Units','O3 Units','SO2 Units','CO Units'],axis=1)

In [8]:
data.head()

,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Mean,NO2 1st Max Value,...,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,...,10,34,3.000000,9.0,21,13.0,1.145833,4.2,21,NaN
1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,...,10,34,3.000000,9.0,21,13.0,0.878947,2.2,23,25.0
2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,...,10,34,2.975000,6.6,23,NaN,1.145833,4.2,21,NaN
3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,...,10,34,2.975000,6.6,23,NaN,0.878947,2.2,23,25.0
4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,22.958333,36.0,...,10,27,1.958333,3.0,22,4.0,0.850000,1.6,23,NaN


#### I also feel that I should drop the following columns, but I am keping those for now
    - Country Code, State Code, Site Num
        - These are just identifiers and have no significance.
        - But for now I am keeping these because I think we can use them as id selectors for selections using d3.
        
    - Address
        - This is the address of the recording station.
        - I am keeping this as I think we can pin point these locations on the map and hovering over them would get you an infobox with the details of the selected recording station.
        - But this will be an optional feature!

## Now, let's check NaN values in the dataset

In [9]:
data.isnull().sum()

State Code                0
County Code               0
Site Num                  0
Address                   0
State                     0
County                    0
City                      0
Date Local                0
NO2 Mean                  0
NO2 1st Max Value         0
NO2 1st Max Hour          0
NO2 AQI                   0
O3 Mean                   0
O3 1st Max Value          0
O3 1st Max Hour           0
O3 AQI                    0
SO2 Mean                  0
SO2 1st Max Value         0
SO2 1st Max Hour          0
SO2 AQI              872907
CO Mean                   0
CO 1st Max Value          0
CO 1st Max Hour           0
CO AQI               873323
dtype: int64

* That's strange! Out of the four AQI columns, SO2 AQI and CO AQI have around 872k null values!
* But I am not going to blindly drop these rows!
* Because that would be a loss of so much of data.
* I'll now come up with a strategy to fill up these values.

#### I have noticed another strange thing about this dataset
* There are 4 rows for each day for each City. 
* It seems as if the data recording had some problem.
* One of the rows has no None values.
* But the values for SO2 and CO statistics are different for them!
* Maybe these are updated in steps! Idk. I'll have to research on this.
* Then I can keep the most recently updated value and drop the others.
* For now, I am assuming that this is the correct row and I'll "blindly" drop the other three rows.

In [10]:
data[(data['Date Local'] == '2000-01-01') & (data['City'] == 'Phoenix')]

,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Mean,NO2 1st Max Value,...,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,...,10,34,3.000,9.0,21,13.0,1.145833,4.2,21,NaN
1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,...,10,34,3.000,9.0,21,13.0,0.878947,2.2,23,25.0
2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,...,10,34,2.975,6.6,23,NaN,1.145833,4.2,21,NaN
3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,...,10,34,2.975,6.6,23,NaN,0.878947,2.2,23,25.0


In [11]:
data.shape

(1746661, 24)

The original no of rows are 1746661. By my logic they shoudld be close to 1746661/4 = 436665.25  after dropping Nan values.

In [12]:
data = data.dropna() 

In [13]:
data.shape

(436876, 24)

### Mexico data?

There is data for Mexico too! But we're not interested in that. So, probably I am going to drop the mexico rows.

In [14]:
data = data[data['State'] != 'Country Of Mexico']

In [15]:
data.shape

(434493, 24)

### Resetting Index to start the rolling process

In [16]:
data.reset_index(drop=True, inplace=True)

## Rolling Up

In [17]:
data1 = data.copy()

#### Converting string date to datetime format

In [18]:
data1['Date Local'] = pd.to_datetime(data1['Date Local'])

#### Setting date as index - for rolling to work

In [19]:
data1.set_index('Date Local', inplace=True)

#### Sampling the data

In [20]:
data1.columns

Index(['State Code', 'County Code', 'Site Num', 'Address', 'State', 'County',
       'City', 'NO2 Mean', 'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI',
       'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Mean',
       'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI', 'CO Mean',
       'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI'],
      dtype='object')

In [21]:
roll_up_map = {
    
    'State Code':'first',
    'County Code':'first',
    'Site Num':'first',
    'Address':'first',
#     'State':'first',
#     'County':'first',
#     'City':'first',
    'NO2 Mean':'mean',
    'NO2 1st Max Value':'max',
    'NO2 1st Max Hour':'mean',
    'NO2 AQI':'mean',
    'O3 Mean':'mean',
    'O3 1st Max Value':'max',
    'O3 1st Max Hour':'mean',
    'O3 AQI':'mean',
    'SO2 Mean':'mean',
    'SO2 1st Max Value':'max',
    'SO2 1st Max Hour':'mean',
    'SO2 AQI':'mean',
    'CO Mean':'mean',
    'CO 1st Max Value':'max',
    'CO 1st Max Hour':'mean',
    'CO AQI':'mean'
}

In [22]:
data1 = data1.groupby(['State','County','City']).resample('M').agg(roll_up_map)

In [23]:
data1.reset_index(drop=False, inplace=True)

In [24]:
data1.set_index('Date Local', inplace=True)
data1.index = data1.index - pd.to_timedelta(data1.index.day - 1, unit='d')

In [25]:
data1.reset_index(drop=False, inplace=True)

### Treatment of Max Hour Columns

##### Now, because the Hour can only be an integer, I'll round the floating point nos to integers.

In [26]:
data1['NO2 1st Max Hour'] = round(data1['NO2 1st Max Hour'])
data1['O3 1st Max Hour'] = round(data1['O3 1st Max Hour'])
data1['SO2 1st Max Hour'] = round(data1['SO2 1st Max Hour'])
data1['CO 1st Max Hour'] = round(data1['CO 1st Max Hour'])

In [27]:
data1.to_csv('data/pollution_us_2000_2016_rolled_up.csv')

## Validation of Results

In [28]:
data1.isnull().sum()

Date Local              0
State                   0
County                  0
City                    0
State Code           2400
County Code          2400
Site Num             2400
Address              2400
NO2 Mean             2400
NO2 1st Max Value    2400
NO2 1st Max Hour     2400
NO2 AQI              2400
O3 Mean              2400
O3 1st Max Value     2400
O3 1st Max Hour      2400
O3 AQI               2400
SO2 Mean             2400
SO2 1st Max Value    2400
SO2 1st Max Hour     2400
SO2 AQI              2400
CO Mean              2400
CO 1st Max Value     2400
CO 1st Max Hour      2400
CO AQI               2400
dtype: int64

* Data1 has some empty rows! Let's check why is it so?
* My assumption is that there should be no data for that date in the original data.

In [29]:
data[(data['State'] == 'Alabama') & (data['County'] == 'Jefferson') & (data['City']=='Birmingham')]

,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Mean,NO2 1st Max Value,...,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
330221,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2013-12-01,17.208333,39.3,...,9,24,0.313636,1.0,11,1.0,0.266667,0.5,0,6.0
330222,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2013-12-02,20.687500,32.4,...,0,12,0.530000,2.4,11,3.0,0.400000,0.5,0,6.0
330223,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2013-12-03,14.912500,22.4,...,22,11,0.305263,2.3,11,3.0,0.258333,0.3,0,3.0
330224,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2013-12-04,7.825000,19.3,...,8,13,0.131818,1.3,17,1.0,0.116667,0.2,20,2.0
330225,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2013-12-05,8.004762,16.0,...,18,13,0.000000,1.1,0,1.0,0.108333,0.2,19,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428483,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2016-05-27,4.975000,10.1,...,8,50,0.363636,1.5,0,1.0,0.200000,0.2,0,2.0
428484,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2016-05-28,9.516667,27.8,...,10,44,0.736364,2.0,23,3.0,0.225000,0.4,22,5.0
428485,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2016-05-29,10.841667,30.9,...,8,40,1.750000,18.3,7,26.0,0.366667,0.6,3,7.0
428486,1,73,23,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,2016-05-30,12.600000,37.1,...,10,44,1.290000,10.1,23,14.0,0.288889,0.5,2,6.0


* Yeah! My assumption was correct.
* This baby is now ready to be visualized!